# 09. Grand Model Comparison / 대규모 모델 비교

이 노트북은 **10개의 옵션 가격 결정 모델**을 종합적으로 비교 분석합니다.

This notebook provides a comprehensive comparison of **10 option pricing models**.

## 모델 라인업 / Model Roster (The Magnificent Ten)

| Category | Model | Description |
|----------|-------|-------------|
| **Baseline** | 1. Black-Scholes | 가장 단순한 기준선 / Simplest benchmark |
| **SDE (Classic)** | 2. Heston | 확률적 변동성 / Stochastic Volatility |
| | 3. Merton | 점프 확산 / Jump Diffusion |
| | 4. Bates | SV + Jump |
| | 5. SVJJ | SV + Price/Vol Jump |
| **SDE (Modern)** | 6. rBergomi | 거친 변동성 / Rough Volatility |
| **Physics** | 7. Quantum Path Integral | 경로 적분 ⭐ / Path Integral |
| **ML** | 8. XGBoost | Gradient Boosting |
| | 9. LSTM | Recurrent NN |
| **DL+Physics** | 10. Neural SDE | 딥러닝 경로 적분 / Neural Path Integral |

---
## Part 0: Setup / 환경 설정

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Fix OMP Error #15

# =============================================================================
# 0.1 Import Libraries / 라이브러리 임포트
# =============================================================================
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import torch
import time
import warnings
warnings.filterwarnings('ignore')

# Visualization / 시각화
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data / 데이터
import yfinance as yf
from datetime import datetime, timedelta
from scipy.interpolate import griddata
from scipy.optimize import differential_evolution
from scipy.stats import norm

# Custom Engines / 커스텀 엔진
from src.physics_engine import MarketSimulator, RBergomiSimulator
from src.neural_engine import NeuralSDESimulator
from src.quantum_solver import PathIntegralSolver
from src.ml_models import BlackScholesModel, XGBoostOptionModel, LSTMOptionModel

# Device Setup / 장치 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
print('All 10 model engines loaded! / 10개 모델 엔진 로드 완료!')

Using device: cuda
All 10 model engines loaded! / 10개 모델 엔진 로드 완료!


In [2]:
# =============================================================================
# 0.2 Utility Functions / 유틸리티 함수
# =============================================================================

def black_scholes_call(S, K, T, r, sigma):
    """Black-Scholes 콜옵션 가격 계산 / Calculate BS call price."""
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def implied_volatility(price, S, K, T, r, max_iter=100, tol=1e-6):
    """내재변동성 역산 (Newton-Raphson) / Compute implied volatility."""
    if price <= 0 or T <= 0:
        return np.nan
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = black_scholes_call(S, K, T, r, sigma)
        vega = S * np.sqrt(T) * norm.pdf((np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T)))
        if vega < 1e-10:
            return np.nan
        diff = bs_price - price
        if abs(diff) < tol:
            return sigma
        sigma = sigma - diff / vega
        if sigma <= 0:
            return np.nan
    return sigma if 0.01 < sigma < 5.0 else np.nan

print('Utility functions loaded / 유틸리티 함수 로드 완료')

Utility functions loaded / 유틸리티 함수 로드 완료


---
## Part 1: Data Loading / 데이터 로드

In [3]:
# =============================================================================
# 1.1 Fetch SPX Option Data / SPX 옵션 데이터 가져오기
# =============================================================================
ticker = yf.Ticker('SPY')
S0 = ticker.history(period='1d')['Close'].iloc[-1]
r = 0.05  # Risk-free rate / 무위험 이자율

print(f'현재 SPY 가격 / Current SPY Price: ${S0:.2f}')
print(f'무위험 이자율 / Risk-free Rate: {r*100:.1f}%')

expirations = ticker.options[:6]
print(f'\n사용 가능한 만기일 / Available Expirations: {expirations}')

현재 SPY 가격 / Current SPY Price: $681.92
무위험 이자율 / Risk-free Rate: 5.0%

사용 가능한 만기일 / Available Expirations: ('2026-01-02', '2026-01-05', '2026-01-06', '2026-01-07', '2026-01-08', '2026-01-09')


In [4]:
# =============================================================================
# 1.2 Build Option Surface DataFrame / 옵션 표면 데이터프레임 구축
# =============================================================================
today = datetime.now()
data_list = []

for exp_date in expirations:
    try:
        opt_chain = ticker.option_chain(exp_date)
        calls = opt_chain.calls
        exp_datetime = datetime.strptime(exp_date, '%Y-%m-%d')
        T = (exp_datetime - today).days / 365.0
        if T <= 0:
            continue
        calls = calls[(calls['strike'] > S0 * 0.85) & (calls['strike'] < S0 * 1.15)]
        calls = calls[calls['volume'] > 10]
        for _, row in calls.iterrows():
            K = row['strike']
            mid_price = (row['bid'] + row['ask']) / 2
            if mid_price > 0:
                iv = implied_volatility(mid_price, S0, K, T, r)
                if not np.isnan(iv):
                    data_list.append({'strike': K, 'T': T, 'price': mid_price, 'iv': iv, 'moneyness': K / S0})
    except Exception as e:
        print(f'Error: {exp_date}: {e}')

df_surface = pd.DataFrame(data_list)
print(f'\n총 데이터 포인트 / Total Data Points: {len(df_surface)}')
df_surface.head()


총 데이터 포인트 / Total Data Points: 179


,strike,T,price,iv,moneyness
0,640.0,0.008219,42.785,0.438622,0.938527
1,670.0,0.008219,13.010,0.184945,0.982520
2,671.0,0.008219,12.015,0.174779,0.983986
3,675.0,0.008219,8.340,0.149709,0.989852
4,676.0,0.008219,7.430,0.142160,0.991319


---
## Part 2: Model Calibration & Training / 모델 캘리브레이션 및 학습

10개 모델 각각을 시장 데이터에 맞춰 최적화합니다.
Optimize each of the 10 models to fit market data.

In [5]:
# =============================================================================
# 2.0 Common Settings / 공통 설정
# =============================================================================
N_paths = 5000
dt = 0.01

market_strikes = df_surface['strike'].values
market_T = df_surface['T'].values
market_prices = df_surface['price'].values
market_iv = df_surface['iv'].values

calibration_results = {}
execution_times = {}
model_prices_dict = {}  # 각 모델별 예측 가격 저장 / Store predicted prices for each model

print(f'캘리브레이션 설정 / Calibration Settings:')
print(f'  - N_paths: {N_paths}, dt: {dt}')
print(f'  - Data points: {len(market_strikes)}')

캘리브레이션 설정 / Calibration Settings:
  - N_paths: 5000, dt: 0.01
  - Data points: 179


In [6]:
# =============================================================================
# 2.1 Model 1: Black-Scholes (Baseline) / 블랙-숄즈 (기준선)
# =============================================================================
print('='*60)
print('📊 Model 1: Black-Scholes (Baseline)')
print('='*60)

start_time = time.time()
bs_model = BlackScholesModel()

# 단일 만기에 대해 캘리브레이션 / Calibrate on first maturity
first_T = np.unique(market_T)[0]
mask = market_T == first_T
bs_model.calibrate(market_prices[mask], S0, market_strikes[mask], first_T, r)

# 전체 가격 예측 / Predict all prices
bs_prices = np.array([bs_model.price(S0, K, T, r) for K, T in zip(market_strikes, market_T)])
bs_rmse = np.sqrt(np.mean((bs_prices - market_prices) ** 2))

elapsed = time.time() - start_time
execution_times['Black-Scholes'] = elapsed
calibration_results['Black-Scholes'] = {'params': f'sigma={bs_model.sigma:.4f}', 'rmse': bs_rmse}
model_prices_dict['Black-Scholes'] = bs_prices

print(f'✅ Black-Scholes 완료 / Complete')
print(f'   Calibrated sigma: {bs_model.sigma:.4f}')
print(f'   RMSE: {bs_rmse:.6f}, Time: {elapsed:.1f}s')

📊 Model 1: Black-Scholes (Baseline)
✅ Black-Scholes 완료 / Complete
   Calibrated sigma: 0.1177
   RMSE: 0.374378, Time: 0.2s


In [7]:
# =============================================================================
# 2.2 Models 2-5: SDE Models (Heston, Merton, Bates, SVJJ)
# =============================================================================
simulator = MarketSimulator(mu=r, kappa=2.0, theta=0.04, xi=0.3, rho=-0.7, device=device)

def global_calibration_loss(params, model_name, market_strikes, market_T, market_prices, S0, r, dt, num_paths, simulator):
    """글로벌 캘리브레이션 손실 함수 / Global calibration loss."""
    try:
        params_dict = {'mu': r}
        if model_name == 'heston':
            kappa, theta, xi, rho = params
            params_dict.update({'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho, 'jump_lambda': 0, 'jump_mean': 0, 'jump_std': 0})
            val_type = 'heston'
        elif model_name == 'merton':
            sigma, jump_lambda, jump_mean, jump_std = params
            params_dict.update({'kappa': 10.0, 'theta': sigma**2, 'xi': 0.001, 'rho': 0.0, 'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std})
            val_type = 'bates'
        elif model_name == 'bates':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std = params
            params_dict.update({'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho, 'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std})
            val_type = 'bates'
        elif model_name == 'svjj':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean = params
            params_dict.update({'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho, 'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': vol_jump_mean})
            val_type = 'svjj'
        else:
            return 1e9
        if params_dict.get('kappa', 1) < 0 or params_dict.get('theta', 1) < 0:
            return 1e9
    except:
        return 1e9
    try:
        unique_T = np.unique(market_T)
        all_model_prices, all_market_prices = [], []
        for T_val in unique_T:
            mask = market_T == T_val
            strikes_T, prices_T = market_strikes[mask], market_prices[mask]
            S_paths, _ = simulator.simulate(S0=S0, v0=params_dict.get('theta', 0.04), T=T_val, dt=dt, num_paths=num_paths, model_type=val_type, override_params=params_dict)
            S_final = S_paths[:, -1]
            if torch.isnan(S_final).any():
                return 1e9
            S_corr = S_final * (S0 / torch.mean(S_final))
            strikes_gpu = torch.tensor(strikes_T, device=simulator.device).float()
            payoffs = torch.maximum(S_corr.unsqueeze(1) - strikes_gpu, torch.tensor(0.0, device=simulator.device))
            model_prices = (torch.mean(payoffs, dim=0) * np.exp(-r * T_val)).cpu().numpy()
            all_model_prices.extend(model_prices)
            all_market_prices.extend(prices_T)
        return np.sqrt(np.mean((np.array(all_model_prices) - np.array(all_market_prices)) ** 2))
    except:
        return 1e9

model_configs = {
    'Heston': {'bounds': [(0.1, 10), (0.01, 0.5), (0.1, 1.0), (-0.95, 0.0)], 'x0': [2.0, 0.04, 0.3, -0.7]},
    'Merton': {'bounds': [(0.05, 0.5), (0.01, 2.0), (-0.2, 0.1), (0.01, 0.3)], 'x0': [0.2, 0.5, -0.05, 0.1]},
    'Bates': {'bounds': [(0.1, 10), (0.01, 0.5), (0.1, 1.0), (-0.95, 0.0), (0.01, 2.0), (-0.2, 0.1), (0.01, 0.3)], 'x0': [2.0, 0.04, 0.3, -0.7, 0.5, -0.05, 0.1]},
    'SVJJ': {'bounds': [(0.1, 10), (0.01, 0.5), (0.1, 1.0), (-0.95, 0.0), (0.01, 2.0), (-0.2, 0.1), (0.01, 0.3), (0.01, 0.2)], 'x0': [2.0, 0.04, 0.3, -0.7, 0.5, -0.05, 0.1, 0.05]}
}
de_options = {'maxiter': 15, 'popsize': 6, 'tol': 0.05, 'disp': True, 'workers': 1}

for model_name, config in model_configs.items():
    print(f'\n{"="*60}\n📊 Model: {model_name}\n{"="*60}')
    start_time = time.time()
    result = differential_evolution(func=global_calibration_loss, bounds=config['bounds'],
        args=(model_name.lower(), market_strikes, market_T, market_prices, S0, r, dt, N_paths, simulator), **de_options)
    elapsed = time.time() - start_time
    execution_times[model_name] = elapsed
    calibration_results[model_name] = {'params': result.x, 'rmse': result.fun}
    print(f'✅ {model_name} 완료, RMSE: {result.fun:.6f}, Time: {elapsed:.1f}s')


📊 Model: Heston
differential_evolution step 1: f(x)= 0.8139870387483072
differential_evolution step 2: f(x)= 0.7903476301208529
differential_evolution step 3: f(x)= 0.7903476301208529
differential_evolution step 4: f(x)= 0.7903476301208529
differential_evolution step 5: f(x)= 0.7873439759000782
differential_evolution step 6: f(x)= 0.7867501369650092
differential_evolution step 7: f(x)= 0.7867501369650092
Polishing solution with 'L-BFGS-B'
✅ Heston 완료, RMSE: 0.786750, Time: 3.8s

📊 Model: Merton
differential_evolution step 1: f(x)= 0.7706477582481734
differential_evolution step 2: f(x)= 0.7316494592047561
differential_evolution step 3: f(x)= 0.6974447043010394
differential_evolution step 4: f(x)= 0.6779236853547239
differential_evolution step 5: f(x)= 0.6745128888122446
differential_evolution step 6: f(x)= 0.6701998498660654
differential_evolution step 7: f(x)= 0.6415360317723512
differential_evolution step 8: f(x)= 0.6415360317723512
differential_evolution step 9: f(x)= 0.635941889823

In [8]:
# =============================================================================
# 2.3 Model 6: rBergomi (Rough Volatility) / 거친 변동성
# =============================================================================
print(f'\n{"="*60}\n📊 Model 6: rBergomi (Rough Volatility)\n{"="*60}')

def rbergomi_loss(params, market_strikes, market_T, market_prices, S0, r, dt, num_paths):
    H, eta, xi, rho = params
    try:
        sim = RBergomiSimulator(H=H, eta=eta, xi=xi, rho=rho, device=device)
        unique_T = np.unique(market_T)
        all_model_prices, all_market_prices = [], []
        for T_val in unique_T:
            mask = market_T == T_val
            strikes_T, prices_T = market_strikes[mask], market_prices[mask]
            S_paths, _ = sim.simulate(S0=S0, T=T_val, dt=dt, num_paths=num_paths, mu=r)
            S_final = S_paths[:, -1]
            if torch.isnan(S_final).any():
                return 1e9
            S_corr = S_final * (S0 / torch.mean(S_final))
            strikes_gpu = torch.tensor(strikes_T, device=device).float()
            payoffs = torch.maximum(S_corr.unsqueeze(1) - strikes_gpu, torch.tensor(0.0, device=device))
            model_prices = (torch.mean(payoffs, dim=0) * np.exp(-r * T_val)).cpu().numpy()
            all_model_prices.extend(model_prices)
            all_market_prices.extend(prices_T)
        return np.sqrt(np.mean((np.array(all_model_prices) - np.array(all_market_prices)) ** 2))
    except:
        return 1e9

start_time = time.time()
result_rbergomi = differential_evolution(func=rbergomi_loss, bounds=[(0.01, 0.4), (0.5, 3.0), (0.05, 0.5), (-0.99, -0.1)],
    args=(market_strikes, market_T, market_prices, S0, r, dt, N_paths), **de_options)
elapsed = time.time() - start_time
execution_times['rBergomi'] = elapsed
calibration_results['rBergomi'] = {'params': result_rbergomi.x, 'rmse': result_rbergomi.fun}
print(f'✅ rBergomi 완료, H={result_rbergomi.x[0]:.4f}, RMSE: {result_rbergomi.fun:.6f}, Time: {elapsed:.1f}s')


📊 Model 6: rBergomi (Rough Volatility)
differential_evolution step 1: f(x)= 2.170227553919935
differential_evolution step 2: f(x)= 1.4649851426024094
differential_evolution step 3: f(x)= 1.4649851426024094
differential_evolution step 4: f(x)= 1.357465391614053
differential_evolution step 5: f(x)= 1.357465391614053
differential_evolution step 6: f(x)= 1.330539303064214
differential_evolution step 7: f(x)= 1.330539303064214
differential_evolution step 8: f(x)= 1.3070493028509782
differential_evolution step 9: f(x)= 1.3070493028509782
differential_evolution step 10: f(x)= 1.3070493028509782
differential_evolution step 11: f(x)= 1.2631303667651268
differential_evolution step 12: f(x)= 1.250842354082319
differential_evolution step 13: f(x)= 1.250842354082319
differential_evolution step 14: f(x)= 1.250842354082319
Polishing solution with 'L-BFGS-B'
✅ rBergomi 완료, H=0.3992, RMSE: 1.250842, Time: 7.6s


In [9]:
# =============================================================================
# 2.4 Model 7: Quantum Path Integral ⭐ / 양자 경로 적분 ⭐
# =============================================================================
print(f'\n{"="*60}\n📊 Model 7: Quantum Path Integral ⭐\n{"="*60}')

start_time = time.time()

# Quantum Solver uses Heston simulator as base
quantum_simulator = MarketSimulator(mu=r, kappa=2.0, theta=0.04, xi=0.3, rho=-0.7, device=device)
quantum_solver = PathIntegralSolver(quantum_simulator)

# Compute prices using Path Integral / 경로 적분으로 가격 계산
quantum_prices = []
for K, T in zip(market_strikes, market_T):
    price = quantum_solver.price_option(S0, K, T, r, num_paths=N_paths, dt=dt)
    quantum_prices.append(price)

quantum_prices = np.array(quantum_prices)
quantum_rmse = np.sqrt(np.nanmean((quantum_prices - market_prices) ** 2))

elapsed = time.time() - start_time
execution_times['Quantum'] = elapsed
calibration_results['Quantum'] = {'params': 'Path Integral Weighting', 'rmse': quantum_rmse}
model_prices_dict['Quantum'] = quantum_prices

print(f'✅ Quantum Path Integral 완료')
print(f'   RMSE: {quantum_rmse:.6f}, Time: {elapsed:.1f}s')


📊 Model 7: Quantum Path Integral ⭐
✅ Quantum Path Integral 완료
   RMSE: 0.784911, Time: 0.6s


In [10]:
# =============================================================================
# 2.5 Model 8: XGBoost / XGBoost 머신러닝
# =============================================================================
print(f'\n{"="*60}\n📊 Model 8: XGBoost\n{"="*60}')

start_time = time.time()

# Feature engineering / 특성 공학
X_train = np.column_stack([df_surface['moneyness'].values, df_surface['T'].values, np.full(len(df_surface), r)])
y_train = market_prices

xgb_model = XGBoostOptionModel()
xgb_model.train(X_train, y_train)
xgb_prices = xgb_model.predict(X_train)
xgb_rmse = np.sqrt(np.mean((xgb_prices - market_prices) ** 2))

elapsed = time.time() - start_time
execution_times['XGBoost'] = elapsed
calibration_results['XGBoost'] = {'params': 'n_estimators=100', 'rmse': xgb_rmse}
model_prices_dict['XGBoost'] = xgb_prices

print(f'✅ XGBoost 완료, RMSE: {xgb_rmse:.6f}, Time: {elapsed:.1f}s')


📊 Model 8: XGBoost
✅ XGBoost 완료, RMSE: 0.035771, Time: 0.6s


In [11]:
# =============================================================================
# 2.6 Model 9: LSTM / LSTM 시계열
# =============================================================================
print(f'\n{"="*60}\n📊 Model 9: LSTM\n{"="*60}')

start_time = time.time()

lstm_model = LSTMOptionModel(input_dim=3, hidden_dim=64, num_layers=2, device=device)
lstm_model.train_model(X_train, y_train, epochs=30, lr=0.001)
lstm_prices = lstm_model.predict(X_train)
lstm_rmse = np.sqrt(np.mean((lstm_prices - market_prices) ** 2))

elapsed = time.time() - start_time
execution_times['LSTM'] = elapsed
calibration_results['LSTM'] = {'params': 'hidden=64, layers=2', 'rmse': lstm_rmse}
model_prices_dict['LSTM'] = lstm_prices

print(f'✅ LSTM 완료, RMSE: {lstm_rmse:.6f}, Time: {elapsed:.1f}s')


📊 Model 9: LSTM
LSTM Epoch 10/30, Loss: 46.167164
LSTM Epoch 20/30, Loss: 45.616680
LSTM Epoch 30/30, Loss: 44.870743
✅ LSTM 완료, RMSE: 6.691750, Time: 2.4s


In [12]:
# =============================================================================
# 2.7 Model 10: Neural SDE / Neural SDE (딥러닝 경로 적분)
# =============================================================================
print(f'\n{"="*60}\n📊 Model 10: Neural SDE (Neural Path Integral)\n{"="*60}')

start_time = time.time()

neural_sde = NeuralSDESimulator(hidden_dim=64, n_layers=3, device=device)
optimizer = torch.optim.Adam(list(neural_sde.drift_net.parameters()) + list(neural_sde.diff_net.parameters()), lr=0.001)

n_epochs = 30
unique_T = np.unique(market_T)
for epoch in range(n_epochs):
    epoch_loss = 0
    for T_val in unique_T[:3]:
        mask = market_T == T_val
        loss = neural_sde.train_step(market_prices[mask], market_strikes[mask], T_val, S0, r, optimizer)
        epoch_loss += loss
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/3:.6f}')

# Final RMSE / 최종 RMSE
neural_sde.drift_net.eval()
neural_sde.diff_net.eval()
all_model_prices, all_market_prices = [], []
with torch.no_grad():
    for T_val in unique_T:
        mask = market_T == T_val
        S_paths, _ = neural_sde.simulate(S0, T_val, dt, N_paths)
        S_final = S_paths[:, -1]
        S_corr = S_final * (S0 / torch.mean(S_final))
        strikes_gpu = torch.tensor(market_strikes[mask], device=device).float()
        payoffs = torch.maximum(S_corr.unsqueeze(1) - strikes_gpu, torch.tensor(0.0, device=device))
        model_prices = (torch.mean(payoffs, dim=0) * np.exp(-r * T_val)).cpu().numpy()
        all_model_prices.extend(model_prices)
        all_market_prices.extend(market_prices[mask])

neural_rmse = np.sqrt(np.nanmean((np.array(all_model_prices) - np.array(all_market_prices)) ** 2))
elapsed = time.time() - start_time
execution_times['Neural SDE'] = elapsed
calibration_results['Neural SDE'] = {'params': 'Neural Network', 'rmse': neural_rmse}

print(f'✅ Neural SDE 완료, RMSE: {neural_rmse:.6f}, Time: {elapsed:.1f}s')


📊 Model 10: Neural SDE (Neural Path Integral)
Epoch 10/30, Loss: 4.617431
Epoch 20/30, Loss: 0.347276
Epoch 30/30, Loss: 0.090244
✅ Neural SDE 완료, RMSE: 0.963091, Time: 2.4s


---
## Part 3: Quantitative Scorecard / 정량 평가

In [14]:
# =============================================================================
# 3.1 Final Leaderboard / 최종 순위표
# =============================================================================
print('='*60)
print('🏆 GRAND MODEL COMPARISON LEADERBOARD / 대규모 모델 비교 순위표')
print('='*60)

results_df = pd.DataFrame([
    {'Model': name, 'RMSE': data['rmse'], 'Time (s)': execution_times.get(name, 0)}
    for name, data in calibration_results.items()
]).sort_values(by='RMSE')

results_df['Rank'] = range(1, len(results_df) + 1)
results_df = results_df[['Rank', 'Model', 'RMSE', 'Time (s)']]

display(results_df.style.background_gradient(cmap='Greens_r', subset=['RMSE']))

🏆 GRAND MODEL COMPARISON LEADERBOARD / 대규모 모델 비교 순위표


,Rank,Model,RMSE,Time (s)
7,1,XGBoost,0.035771,0.560869
0,2,Black-Scholes,0.374378,0.202850
4,3,SVJJ,0.624785,13.200636
3,4,Bates,0.628819,10.406048
2,5,Merton,0.635942,4.703428
6,6,Quantum,0.784911,0.583345
1,7,Heston,0.786750,3.837411
9,8,Neural SDE,0.963091,2.382590
5,9,rBergomi,1.250842,7.565472
8,10,LSTM,6.691750,2.417364


---
## Part 4: Static Visualization / 정적 시각화 (2D/3D Model Analysis)

In [19]:
# =============================================================================
# 4.1 2D Smile Curves (Slices) / 2D 스마일 커브 (단면 분석)
# =============================================================================
def plot_2d_smile_comparison(market_strikes, market_imvol, market_T, model_prices_dict, S0, r, target_T_idx=0):
    unique_T = np.unique(market_T)
    if target_T_idx >= len(unique_T):
        target_T_idx = 0
    
    target_T = unique_T[target_T_idx]
    mask = market_T == target_T
    
    strikes = market_strikes[mask]
    market_iv_slice = market_imvol[mask]
    
    fig = go.Figure()
    
    # Market Data
    fig.add_trace(go.Scatter(x=strikes, y=market_iv_slice, mode='markers', name='Market', marker=dict(size=10, color='black', symbol='x')))
    
    # Models
    colors = ['blue', 'green', 'red', 'orange', 'purple', 'cyan', 'magenta', 'lime', 'brown', 'teal']
    for i, (model_name, prices) in enumerate(model_prices_dict.items()):
        model_prices_slice = prices[mask]
        # Calculate Implied Vol for Model Prices
        model_iv = []
        for p, k in zip(model_prices_slice, strikes):
             iv = implied_volatility(p, S0, k, target_T, r)
             model_iv.append(iv)
        
        fig.add_trace(go.Scatter(x=strikes, y=model_iv, mode='lines', name=model_name, line=dict(color=colors[i % len(colors)], width=2)))

    fig.update_layout(
        title=f'Volatility Smile Comparison (T={target_T:.3f})',
        xaxis_title='Strike Price',
        yaxis_title='Implied Volatility',
        template='plotly_white',
        width=900, height=600
    )
    fig.show()

# Plot for the first and last maturity
plot_2d_smile_comparison(market_strikes, market_iv, market_T, model_prices_dict, S0, r, target_T_idx=0)
plot_2d_smile_comparison(market_strikes, market_iv, market_T, model_prices_dict, S0, r, target_T_idx=-1)

In [16]:
# =============================================================================
# 4.2 3D Implied Volatility Surface / 3D 내재변동성 표면
# =============================================================================
# Grid for Surface
grid_K, grid_T = np.meshgrid(
    np.linspace(market_strikes.min(), market_strikes.max(), 30),
    np.linspace(market_T.min(), market_T.max(), 30)
)

# Interpolate Market IV
grid_IV = griddata((market_strikes, market_T), market_iv, (grid_K, grid_T), method='cubic')

fig = go.Figure(data=[go.Surface(x=grid_K, y=grid_T, z=grid_IV, colorscale='Viridis', opacity=0.8, name='Market Surface')])

# Add Scatter Points
fig.add_trace(go.Scatter3d(
    x=market_strikes, y=market_T, z=market_iv,
    mode='markers', marker=dict(size=3, color='red'), name='Market Data'
))

fig.update_layout(
    title='Market Implied Volatility Surface',
    scene=dict(xaxis_title='Strike', yaxis_title='Time to Maturity', zaxis_title='Implied Volatility'),
    width=900, height=700,
    template='plotly_dark' # Dark mode for better 3D visibility
)
fig.show()

---
## Part 5: 4D Spacetime Visualization / 4D 시공간 시각화

Quantum Path Integral vs Neural SDE: Path Distribution Analysis.
양자 경로 적분과 뉴럴 SDE의 "경로 분포"를 시각적으로 비교합니다.

In [17]:
# =============================================================================
# 5.1 Generate Paths for Visualization / 시각화를 위한 경로 생성
# =============================================================================
viz_paths = 100  # Number of paths to visualize (keep it small for rendering)
viz_T = 1.0      # Visualization horizon

# 1. Quantum (Heston-based) Paths
sim_quantum = MarketSimulator(mu=r, kappa=2.0, theta=0.04, xi=0.3, rho=-0.7, device=device)
S_quant, v_quant = sim_quantum.simulate(S0, 0.04, viz_T, dt, viz_paths, 'heston')
S_quant = S_quant.cpu().numpy()

# 2. Neural SDE Paths
neural_sde.drift_net.eval()
neural_sde.diff_net.eval()
with torch.no_grad():
    S_neural, _ = neural_sde.simulate(S0, viz_T, dt, viz_paths)
    S_neural = S_neural.cpu().numpy()

time_steps = np.linspace(0, viz_T, S_quant.shape[1])
print("Paths generated for visualization.")

Paths generated for visualization.


In [18]:
# =============================================================================
# 5.2 4D Path Visualizer / 4D 경로 시각화기
# =============================================================================
def plot_spacetime_paths(S_paths, title, color_theme='Blues'):
    fig = go.Figure()
    
    # Plot individual paths
    for i in range(min(50, S_paths.shape[0])): # Limit to 50 paths
        fig.add_trace(go.Scatter3d(
            x=time_steps, 
            y=np.full_like(time_steps, i), # Path Index on Y-axis
            z=S_paths[i, :],
            mode='lines',
            line=dict(width=2, color=S_paths[i, :], colorscale=color_theme),
            name=f'Path {i}'
        ))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='Time (t)',
            yaxis_title='Path Index (Microstate)',
            zaxis_title='Price (S_t)'
        ),
        width=900, height=700,
        template='plotly_dark',
        showlegend=False
    )
    fig.show()

print("Visualizing Quantum Path Integral Microstates...")
plot_spacetime_paths(S_quant, "Quantum Path Integral: Feynman Paths", 'Viridis')

print("Visualizing Neural SDE Microstates...")
plot_spacetime_paths(S_neural, "Neural SDE: Learned Stochastic Paths", 'Plasma')

Visualizing Quantum Path Integral Microstates...


Visualizing Neural SDE Microstates...


---
## Conclusion / 결론

**1. Performance (RMSE):** XGBoost showed the best fit, highlighting ML's interpolation power.
**2. Physics (Consistency):** Heston/Quantum models provided robust, explainable outcomes.
**3. Innovation (Neural SDE):** Successfully demonstrated a hybrid Neural-SDE approach creating realistic paths from scratch.

This concludes the Grand Model Comparison.